In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

data = pd.read_csv("Robust_scaller_result.csv",encoding="cp949")
data

,CUSTNO,GNO,CBSCORE,CBSCOREGRD,CREDITOTAMT,YSALEAMT,ESTMM,ASSETAMT,IMSAAMT,IMJUAMT,...,KIND_JOB_K,KIND_JOB_L,KIND_JOB_M,KIND_JOB_N,KIND_JOB_O,KIND_JOB_P,KIND_JOB_Q,KIND_JOB_R,KIND_JOB_S,KIND_JOB_T
0,475821,l180202101898,1.015873,-0.5,0.230769,10.743169,0.014085,-0.176471,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,74417,l230201700120,0.206349,0.0,0.205128,6.027322,-0.380282,-0.088235,-0.5,25.0,...,0,0,0,0,0,0,0,0,0,0
2,387787,l110201603233,0.396825,-0.5,-0.282051,3.775956,2.098592,-0.176471,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,395418,l230201700254,0.000000,0.0,0.076923,3.562842,-0.352113,-0.176471,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,190372,l200201601418,-2.476190,2.0,-0.102564,1.437158,1.845070,0.235294,-0.4,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39112,514919,l240201902522,-0.253968,0.5,-0.282051,-0.393443,-0.338028,-0.235294,-0.5,0.0,...,0,0,0,0,0,0,0,0,0,0
39113,357361,l160201601877,-1.603175,1.5,0.000000,-0.393443,-0.366197,-0.176471,-0.5,10.0,...,0,0,0,0,0,1,0,0,0,0
39114,340606,l270201900572,-2.349206,2.0,-0.051282,-0.393443,0.140845,-0.235294,-0.5,0.0,...,0,0,0,0,0,0,0,0,1,0
39115,505207,l190201902523,1.063492,-0.5,-0.282051,-0.393443,-0.436620,-0.235294,-0.5,0.0,...,0,0,0,0,0,1,0,0,0,0


In [2]:
features = data[['CBSCORE', 'CREDITOTAMT', 'YSALEAMT',
       'ESTMM',  'IMSAAMT', 'IMJUAMT', 'BUJUAMT',
       'BU1TOTAMT', 'GAMT', 'LABORCNT', 'KOSPI', '환율', 'GDP', 
       '실업률', '물가지수', '국고채', '금리', '유가등락률',
       '소비자심리지수','KIND_JOB_A', 'KIND_JOB_B', 'KIND_JOB_C', 'KIND_JOB_D',
       'KIND_JOB_E', 'KIND_JOB_F', 'KIND_JOB_G', 'KIND_JOB_H', 'KIND_JOB_I',
       'KIND_JOB_J', 'KIND_JOB_K', 'KIND_JOB_L', 'KIND_JOB_M', 'KIND_JOB_N',
       'KIND_JOB_O', 'KIND_JOB_P', 'KIND_JOB_Q', 'KIND_JOB_R', 'KIND_JOB_S',
       'KIND_JOB_T']]
act = data['ACTCD']

In [3]:
from sklearn.model_selection import train_test_split

train_x, test_x,train_y, test_y = train_test_split(features, act,
stratify=act,train_size=0.7,test_size=0.3,random_state=1)

print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(27381, 39) (11736, 39) (27381,) (11736,)


In [4]:
from imblearn.over_sampling import BorderlineSMOTE, ADASYN
from imblearn.under_sampling import EditedNearestNeighbours, TomekLinks
from imblearn.combine import SMOTEENN, SMOTETomek

#언더 샘플링
enn = EditedNearestNeighbours(kind_sel="all", n_neighbors=10)
tomekl = TomekLinks()

#오버 샘플링
bsmote = BorderlineSMOTE(random_state=42)
adasyn = ADASYN(random_state=42)

#혼합 샘플링
smotee = SMOTEENN(random_state=42)
smoteT = SMOTETomek(random_state=42)


X_under1_train, Y_under1_train = enn.fit_resample(train_x, train_y)
X_under2_train, Y_under2_train = tomekl.fit_resample(train_x, train_y)

X_over1_train, Y_over1_train = bsmote.fit_resample(train_x,train_y)
X_over2_train, Y_over2_train = adasyn.fit_resample(train_x,train_y)

X_comb1_train, Y_comb1_train = smotee.fit_resample(train_x, train_y)
X_comb2_train, Y_comb2_train = smoteT.fit_resample(train_x, train_y)

In [5]:
def cut_off(y, threshold) :
    Y = y.copy()  # 대문자 Y를 새로운 변수로 하여 기존의 y값에 영향이 가지 않도록 한다.
    Y[Y>threshold] = 1
    Y[Y<threshold] = 0
    return Y.astype(int)

from sklearn.metrics import confusion_matrix
# confusion matrix accuracy(정확도) 계산함수

def acc(cfmat):
    return (cfmat[0,0] + cfmat[1,1])/(cfmat[0,0] + cfmat[1,1] + cfmat[0,1] + cfmat[1,0])

# 오버샘플링

1. BorderlineSMOTE

In [6]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [7]:
eclf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=0.1)

In [8]:
from sklearn.model_selection import cross_val_score
cross_val_score(eclf, X_over1_train, Y_over1_train, cv=5).mean()

0.8520795660036168

In [9]:
from sklearn.tree import DecisionTreeClassifier
DecisionTreeClassifier()

DecisionTreeClassifier()

In [10]:
AdaBoostClassifier()

AdaBoostClassifier()

In [11]:
params = {"base_estimator__criterion" : ["gini", "entropy"],
          "base_estimator__max_features" : [7,8,],
          "base_estimator__max_depth" : [1,2],
          "n_estimators": [23,24, 25, 26, 27],
          "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
         }

In [12]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X_over1_train, Y_over1_train)

grid.best_score_
grid.best_params_

{'base_estimator__criterion': 'entropy',
 'base_estimator__max_depth': 2,
 'base_estimator__max_features': 8,
 'learning_rate': 0.6,
 'n_estimators': 27}

In [13]:
grid.best_estimator_.feature_importances_

array([0.06108178, 0.02927182, 0.02197669, 0.05148614, 0.08463322,
       0.02855829, 0.05180733, 0.00786906, 0.02921459, 0.05675017,
       0.00753957, 0.00567278, 0.04966908, 0.03327204, 0.0150652 ,
       0.01618585, 0.01681403, 0.02103825, 0.02260997, 0.        ,
       0.        , 0.04184683, 0.        , 0.        , 0.01652248,
       0.06040004, 0.02914651, 0.06099815, 0.        , 0.        ,
       0.01614006, 0.03182987, 0.01643101, 0.        , 0.04108636,
       0.        , 0.02972445, 0.0453584 , 0.        ])

In [14]:
from sklearn.metrics import precision_score,recall_score,f1_score,roc_auc_score

dtc = DecisionTreeClassifier(criterion = 'entropy', max_depth = 2, max_features = 8)
abc = AdaBoostClassifier(learning_rate = 0.55, n_estimators = 26)
model = abc.fit(X_over1_train, Y_over1_train)

pred_y = model.predict(test_x)
pred_Y = cut_off(pred_y, 0.5)

In [15]:
cfmat = confusion_matrix(test_y,pred_Y)
cfmat

array([[8183, 2483],
       [ 488,  582]], dtype=int64)

In [16]:
from sklearn.metrics import classification_report
print(classification_report(test_y, pred_Y, target_names=['사고보증', '정상보증']))

              precision    recall  f1-score   support

        사고보증       0.94      0.77      0.85     10666
        정상보증       0.19      0.54      0.28      1070

    accuracy                           0.75     11736
   macro avg       0.57      0.66      0.56     11736
weighted avg       0.87      0.75      0.79     11736



In [17]:
accuracy = acc(cfmat)
accuracy

0.7468473074301295

In [18]:
pr = precision_score(test_y,pred_Y)
r = recall_score(test_y,pred_Y)
f1= f1_score(test_y,pred_Y)
auc_score = roc_auc_score(test_y,pred_Y)

print("precision: ", pr)
print("recall: ", r)
print("f1 score: ", f1)
print("accuracy: ", auc_score)

precision:  0.1898858075040783
recall:  0.5439252336448598
f1 score:  0.2814993954050786
accuracy:  0.655564716953688


2. ADASYN

In [19]:
eclf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=0.1)
cross_val_score(eclf, X_over2_train, Y_over2_train, cv=5).mean()

0.8648172999674044

In [20]:
DecisionTreeClassifier()
AdaBoostClassifier()

params = {"base_estimator__criterion" : ["gini", "entropy"],
          "base_estimator__max_features" : [7,8,],
          "base_estimator__max_depth" : [1,2],
          "n_estimators": [23,24, 25, 26, 27],
          "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
         }

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X_over2_train, Y_over2_train)

grid.best_score_
grid.best_params_

{'base_estimator__criterion': 'gini',
 'base_estimator__max_depth': 2,
 'base_estimator__max_features': 7,
 'learning_rate': 0.6,
 'n_estimators': 27}

In [21]:
dtc = DecisionTreeClassifier(criterion = 'entropy', max_depth = 2, max_features = 8)
abc = AdaBoostClassifier(learning_rate = 0.45, n_estimators = 27)
model = abc.fit(X_over2_train, Y_over2_train)

pred_y = model.predict(test_x)
pred_Y = cut_off(pred_y, 0.5)

In [22]:
cfmat = confusion_matrix(test_y,pred_Y)
cfmat

array([[8068, 2598],
       [ 489,  581]], dtype=int64)

In [23]:
from sklearn.metrics import classification_report
print(classification_report(test_y, pred_Y, target_names=['사고보증', '정상보증']))

              precision    recall  f1-score   support

        사고보증       0.94      0.76      0.84     10666
        정상보증       0.18      0.54      0.27      1070

    accuracy                           0.74     11736
   macro avg       0.56      0.65      0.56     11736
weighted avg       0.87      0.74      0.79     11736



In [24]:
accuracy = acc(cfmat)
accuracy

0.7369631901840491

In [25]:
pr = precision_score(test_y,pred_Y)
r = recall_score(test_y,pred_Y)
f1= f1_score(test_y,pred_Y)
auc_score = roc_auc_score(test_y,pred_Y)

print("precision: ", pr)
print("recall: ", r)
print("f1 score: ", f1)
print("accuracy: ", auc_score)

precision:  0.18276187480339728
recall:  0.5429906542056074
f1 score:  0.27347611202635913
accuracy:  0.649706465298941


# 언더샘플링

1. EditedNearestNeighbours

In [26]:
eclf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=0.1)
cross_val_score(eclf, X_under1_train, Y_under1_train, cv=5).mean()

0.865584798419853

In [27]:
DecisionTreeClassifier()
AdaBoostClassifier()

params = {"base_estimator__criterion" : ["gini", "entropy"],
          "base_estimator__max_features" : [7,8,],
          "base_estimator__max_depth" : [1,2],
          "n_estimators": [23,24, 25, 26, 27],
          "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
         }

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X_under1_train, Y_under1_train)

grid.best_score_
grid.best_params_

{'base_estimator__criterion': 'gini',
 'base_estimator__max_depth': 2,
 'base_estimator__max_features': 8,
 'learning_rate': 0.6,
 'n_estimators': 24}

In [28]:
dtc = DecisionTreeClassifier(criterion = 'gini', max_depth = 2, max_features = 7)
abc = AdaBoostClassifier(learning_rate = 0.6, n_estimators = 27)
model = abc.fit(X_under1_train, Y_under1_train)

pred_y = model.predict(test_x)
pred_Y = cut_off(pred_y, 0.5)

In [29]:
cfmat = confusion_matrix(test_y,pred_Y)
cfmat

array([[9800,  866],
       [ 721,  349]], dtype=int64)

In [30]:
from sklearn.metrics import classification_report
print(classification_report(test_y, pred_Y, target_names=['사고보증', '정상보증']))

              precision    recall  f1-score   support

        사고보증       0.93      0.92      0.93     10666
        정상보증       0.29      0.33      0.31      1070

    accuracy                           0.86     11736
   macro avg       0.61      0.62      0.62     11736
weighted avg       0.87      0.86      0.87     11736



In [31]:
accuracy = acc(cfmat)
accuracy

0.8647750511247444

In [32]:
pr = precision_score(test_y,pred_Y)
r = recall_score(test_y,pred_Y)
f1= f1_score(test_y,pred_Y)
auc_score = roc_auc_score(test_y,pred_Y)

print("precision: ", pr)
print("recall: ", r)
print("f1 score: ", f1)
print("accuracy: ", auc_score)

precision:  0.2872427983539095
recall:  0.32616822429906545
f1 score:  0.3054704595185996
accuracy:  0.6224878248815785


2. TomekLinks

In [33]:
eclf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=0.1)
cross_val_score(eclf, X_under2_train, Y_under2_train, cv=5).mean()

0.9065189709449204

In [34]:
DecisionTreeClassifier()
AdaBoostClassifier()

params = {"base_estimator__criterion" : ["gini", "entropy"],
          "base_estimator__max_features" : [7,8,],
          "base_estimator__max_depth" : [1,2],
          "n_estimators": [23,24, 25, 26, 27],
          "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
         }

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X_under2_train, Y_under2_train)

grid.best_score_
grid.best_params_

{'base_estimator__criterion': 'gini',
 'base_estimator__max_depth': 2,
 'base_estimator__max_features': 8,
 'learning_rate': 0.6,
 'n_estimators': 23}

In [35]:
dtc = DecisionTreeClassifier(criterion = 'entropy', max_depth = 2, max_features = 7)
abc = AdaBoostClassifier(learning_rate = 0.55, n_estimators = 25)
model = abc.fit(X_under2_train, Y_under2_train)

pred_y = model.predict(test_x)
pred_Y = cut_off(pred_y, 0.5)

In [36]:
cfmat = confusion_matrix(test_y,pred_Y)
cfmat

array([[10666,     0],
       [ 1070,     0]], dtype=int64)

In [37]:
from sklearn.metrics import classification_report
print(classification_report(test_y, pred_Y, target_names=['사고보증', '정상보증']))

              precision    recall  f1-score   support

        사고보증       0.91      1.00      0.95     10666
        정상보증       0.00      0.00      0.00      1070

    accuracy                           0.91     11736
   macro avg       0.45      0.50      0.48     11736
weighted avg       0.83      0.91      0.87     11736



C:\Users\user\anaconda3\envs\Study_Big\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [38]:
accuracy = acc(cfmat)
accuracy

0.9088275391956373

In [39]:
pr = precision_score(test_y,pred_Y)
r = recall_score(test_y,pred_Y)
f1= f1_score(test_y,pred_Y)
auc_score = roc_auc_score(test_y,pred_Y)

print("precision: ", pr)
print("recall: ", r)
print("f1 score: ", f1)
print("accuracy: ", auc_score)

precision:  0.0
recall:  0.0
f1 score:  0.0
accuracy:  0.5


C:\Users\user\anaconda3\envs\Study_Big\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 혼합샘플링

1. SMOTEENN

In [40]:
eclf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=0.1)
cross_val_score(eclf, X_comb1_train, Y_comb1_train, cv=5).mean()

0.8772321989200049

In [41]:
DecisionTreeClassifier()
AdaBoostClassifier()

params = {"base_estimator__criterion" : ["gini", "entropy"],
          "base_estimator__max_features" : [7,8,],
          "base_estimator__max_depth" : [1,2],
          "n_estimators": [23,24, 25, 26, 27],
          "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
         }

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X_comb1_train, Y_comb1_train)

grid.best_score_
grid.best_params_

{'base_estimator__criterion': 'gini',
 'base_estimator__max_depth': 2,
 'base_estimator__max_features': 8,
 'learning_rate': 0.6,
 'n_estimators': 27}

In [42]:
dtc = DecisionTreeClassifier(criterion = 'gini', max_depth = 2, max_features = 8)
abc = AdaBoostClassifier(learning_rate = 0.6, n_estimators = 24)
model = abc.fit(X_comb1_train, Y_comb1_train)

pred_y = model.predict(test_x)
pred_Y = cut_off(pred_y, 0.5)

In [43]:
cfmat = confusion_matrix(test_y,pred_Y)
cfmat

array([[6439, 4227],
       [ 277,  793]], dtype=int64)

In [44]:
from sklearn.metrics import classification_report
print(classification_report(test_y, pred_Y, target_names=['사고보증', '정상보증']))

              precision    recall  f1-score   support

        사고보증       0.96      0.60      0.74     10666
        정상보증       0.16      0.74      0.26      1070

    accuracy                           0.62     11736
   macro avg       0.56      0.67      0.50     11736
weighted avg       0.89      0.62      0.70     11736



In [45]:
accuracy = acc(cfmat)
accuracy

0.6162235855487389

In [46]:
pr = precision_score(test_y,pred_Y)
r = recall_score(test_y,pred_Y)
f1= f1_score(test_y,pred_Y)
auc_score = roc_auc_score(test_y,pred_Y)

print("precision: ", pr)
print("recall: ", r)
print("f1 score: ", f1)
print("accuracy: ", auc_score)

precision:  0.15796812749003983
recall:  0.7411214953271028
f1 score:  0.2604269293924466
accuracy:  0.6724077381004538


2. SMOTETomek

In [47]:
eclf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=0.1)
cross_val_score(eclf, X_comb2_train, Y_comb2_train, cv=5).mean()

0.8577848247735901

In [48]:
DecisionTreeClassifier()
AdaBoostClassifier()

params = {"base_estimator__criterion" : ["gini", "entropy"],
          "base_estimator__max_features" : [7,8,],
          "base_estimator__max_depth" : [1,2],
          "n_estimators": [23,24, 25, 26, 27],
          "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
         }

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X_comb2_train, Y_comb2_train)

grid.best_score_
grid.best_params_

{'base_estimator__criterion': 'entropy',
 'base_estimator__max_depth': 2,
 'base_estimator__max_features': 8,
 'learning_rate': 0.6,
 'n_estimators': 27}

In [49]:
dtc = DecisionTreeClassifier(criterion = 'entropy', max_depth = 2, max_features = 8)
abc = AdaBoostClassifier(learning_rate = 0.55, n_estimators = 27)
model = abc.fit(X_comb2_train, Y_comb2_train)

pred_y = model.predict(test_x)
pred_Y = cut_off(pred_y, 0.5)

In [50]:
cfmat = confusion_matrix(test_y,pred_Y)
cfmat

array([[8146, 2520],
       [ 554,  516]], dtype=int64)

In [51]:
from sklearn.metrics import classification_report
print(classification_report(test_y, pred_Y, target_names=['사고보증', '정상보증']))

              precision    recall  f1-score   support

        사고보증       0.94      0.76      0.84     10666
        정상보증       0.17      0.48      0.25      1070

    accuracy                           0.74     11736
   macro avg       0.55      0.62      0.55     11736
weighted avg       0.87      0.74      0.79     11736



In [52]:
accuracy = acc(cfmat)
accuracy

0.7380708929788684

In [53]:
pr = precision_score(test_y,pred_Y)
r = recall_score(test_y,pred_Y)
f1= f1_score(test_y,pred_Y)
auc_score = roc_auc_score(test_y,pred_Y)

print("precision: ", pr)
print("recall: ", r)
print("f1 score: ", f1)
print("accuracy: ", auc_score)

precision:  0.16996047430830039
recall:  0.4822429906542056
f1 score:  0.2513395031660984
accuracy:  0.6229891120531482
